---
title: "Example Daily Report"
title-block-banner: true
title-block-color: red
date: today
execute:
  echo: false
  warning: false
  include: false
author: "MINTS"
format:
  pdf:
    code-fold: true   
jupyter: python3

---

In [1]:
date:"2024-07-01 00:00:00"
node: "vaL Node 01"

In [2]:
# Injected Parameters
node = "vaLo Node 01"
date = "2024-07-01 00:00:00"


In [3]:


from datetime import datetime, timedelta
import datetime
import pandas as pd
import requests
import pytz
from timezonefinder import TimezoneFinder
from io import StringIO
import json
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import font_manager
import seaborn as sns
from astral import LocationInfo
from astral.sun import sun
from astral.sun import elevation
from matplotlib import rcParams
import matplotlib.font_manager as fm
from matplotlib.ticker import MultipleLocator
from matplotlib.offsetbox import OffsetImage, AnnotationBbox

In [4]:
# Define the font path
montserrat_path = "./Montserrat/static"
 
# Add the font directory to the font manager
font_files = font_manager.findSystemFonts(fontpaths=montserrat_path)
for font_file in font_files:
    font_manager.fontManager.addfont(font_file)

In [5]:
# Load credentials; change to the correct address
with open('./credentials.json') as f:
    creds = json.load(f)

token = creds["token"]
org = creds["orgname"]
bucket = creds["bucket"]
url = f"http://mdash.circ.utdallas.edu:8086/api/v2/query?org={org}"

headers = {
    "Authorization": f"Token {token}",
    "Accept": "application/csv",
    "Content-type": "application/vnd.flux"
}


In [6]:
# Set the initial datetime and node

date_str = date
d = datetime.datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S')

#d = datetime.datetime(2024, 7, 1, 0, 0, 0)
node = node

dstart = d - datetime.timedelta(minutes=120)
dend = d + datetime.timedelta(days=1, minutes=25)

In [7]:
# GPS query
query_gps = f"""
from(bucket: "SharedAirDFW")
  |> range(start: time(v: "{d.isoformat()}Z"), stop: time(v: "{(d + datetime.timedelta(days=1)).isoformat()}Z"))
  |> filter(fn: (r) => r["device_name"] == "{node}")
  |> filter(fn: (r) => r["_measurement"] == "GPSGPGGA2")
  |> filter(fn: (r) => r["_field"] == "latitudeCoordinate" or r["_field"] == "longitudeCoordinate" or r["_field"] == "altitude")
  |> aggregateWindow(every: 1m, fn: mean, createEmpty: true)
  |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
  |> keep(columns: ["_time", "latitudeCoordinate", "longitudeCoordinate", "altitude"])
"""
resp = requests.post(url, headers=headers, data=query_gps)
df = pd.read_csv(StringIO(resp.text))
df.drop(columns=['result', 'table', '_time','Unnamed: 0'], inplace=True)
df.dropna(inplace=True)

In [8]:
pos = {
    'lat': df['latitudeCoordinate'].mean(),
    'lon': df['longitudeCoordinate'].mean(),
    'alt': df['altitude'].mean()
}

# Get the timezone at the given latitude and longitude
tf = TimezoneFinder()
tzone = tf.timezone_at(lat=pos['lat'], lng=pos['lon'])

# Set the timezone for the datetime
d = pytz.timezone(tzone).localize(d)
dstart = d - datetime.timedelta(minutes=120)
dend = d + datetime.timedelta(days=1, minutes=25)

In [9]:
def process_df(response, tzone, d):
    df = pd.read_csv(StringIO(response.text))
    df = df.drop(columns=['Unnamed: 0', 'result', 'table'])
    df = df.rename(columns={'_time': 'datetime'})
    df['datetime'] = pd.to_datetime(df['datetime'])
    df['datetime'] = df['datetime'].dt.tz_convert(tzone)
    df = df[df['datetime'].dt.date == d.date()]
    df = df.dropna()
    return df

dstart_str = dstart.isoformat()
dend_str = dend.isoformat()

# Queries
query_ips = f"""
from(bucket: "SharedAirDFW")
  |> range(start: time(v: "{dstart_str}"), stop: time(v: "{dend_str}"))
  |> filter(fn: (r) => r["device_name"] == "{node}")
  |> filter(fn: (r) => r["_measurement"] == "IPS7100")
  |> filter(fn: (r) => r["_field"] == "pm0_1" or r["_field"] == "pm0_3" or r["_field"] == "pm0_5" or r["_field"] == "pm1_0" or r["_field"] == "pm2_5" or r["_field"] == "pm5_0" or r["_field"] == "pm10_0" or r["_field"] == "pc0_1" or r["_field"] == "pc0_3" or r["_field"] == "pc0_5" or r["_field"] == "pc1_0" or r["_field"] == "pc2_5" or r["_field"] == "pc5_0" or r["_field"] == "pc10_0")
  |> aggregateWindow(every: 1m, period: 5m, offset:-150s, fn: mean, createEmpty: true)
  |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
  |> keep(columns: ["_time", "pm0_1", "pm0_3", "pm0_5", "pm1_0", "pm2_5", "pm5_0", "pm10_0","pc0_1", "pc0_3", "pc0_5", "pc1_0", "pc2_5", "pc5_0", "pc10_0"])

  
"""

query_bme = f"""
from(bucket: "SharedAirDFW")
  |> range(start: time(v: "{dstart_str}"), stop: time(v: "{dend_str}"))
  |> filter(fn: (r) => r["device_name"] == "{node}")
  |> filter(fn: (r) => r["_measurement"] == "BME280V2")
  |> filter(fn: (r) => r["_field"] == "temperature" or r["_field"] == "pressure" or r["_field"] == "humidity" or r["_field"] == "dewPoint")
  |> aggregateWindow(every: 1m, period: 5m, offset:-150s, fn: mean, createEmpty: true)
  |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
  |> keep(columns: ["_time", "temperature", "pressure", "humidity", "dewPoint"])
"""

query_rg15 = f"""
from(bucket: "SharedAirDFW")
  |> range(start: time(v: "{dstart_str}"), stop: time(v: "{dend_str}"))
  |> filter(fn: (r) => r["device_name"] == "{node}")
  |> filter(fn: (r) => r["_measurement"] == "RG15")
  |> filter(fn: (r) => r["_field"] == "rainPerInterval" or r["_field"] == "accumulation")
  |> aggregateWindow(every: 1m, period: 5m, offset:-150s, fn: mean, createEmpty: true)
  |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
  |> keep(columns: ["_time", "rainPerInterval"])
"""

query_scd30 = """
from(bucket: "SharedAirDFW")
  |> range(start: time(v: "{dstart_str}"), stop: time(v: "{dend_str}"))
  |> filter(fn: (r) => r["device_name"] == "{node}")
  |> filter(fn: (r) => r["_measurement"] == "SCD30V2")
  |> filter(fn: (r) => r["_field"] == "co2" or r["_field"] == "temperature")
  |> aggregateWindow(every: 1m, period: 5m, offset:-150s,  fn: mean, createEmpty: true)
  |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
  |> keep(columns: ["_time", "co2"])
"""   

query_wim = f"""
from(bucket: "SharedAirDFW")
  |> range(start: time(v: "{dstart_str}"), stop: time(v: "{dend_str}"))
  |> filter(fn: (r) => r["device_name"] == "{node}")
  |> filter(fn: (r) => r["_measurement"] == "WIMWV")
  |> filter(fn: (r) => r["_field"] == "windSpeed" or r["_field"] == "windAngle")
  |> aggregateWindow(every: 1m, period: 5m, offset:-150s, fn: mean, createEmpty: true)
  |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
  |> keep(columns: ["_time", "windSpeed","windAngle"])
"""

# Send requests and process data
resp_ips = requests.post(url, headers=headers, data=query_ips)
df_ips = process_df(resp_ips, tzone, d)

resp_bme = requests.post(url, headers=headers, data=query_bme)
df_bme = process_df(resp_bme, tzone, d)

resp_rg15 = requests.post(url, headers=headers, data=query_rg15)
df_rg = process_df(resp_rg15, tzone, d)

resp_wim = requests.post(url, headers=headers, data=query_wim)
df_wim = process_df(resp_wim, tzone, d)

df = pd.merge(df_ips, df_bme, on='datetime', how='left')
df = pd.merge(df, df_rg, on='datetime', how='left')
df = pd.merge(df, df_wim, on='datetime', how='left')

In [10]:
#Add solar elevation
location = LocationInfo(latitude=pos['lat'], longitude=pos['lon'], timezone='UTC')

def calculate_solar_elevation(timestamp):
    date = timestamp.to_pydatetime()
    return elevation(location.observer, date)

df['sol_el'] = df['datetime'].apply(calculate_solar_elevation)
df['sol_zenith'] = 90 - df['sol_el']

df['rainPerInterval'] = df['rainPerInterval'].fillna(0.0).replace(np.nan, 0.0)
df['dt'] = [(v - df['datetime'].iloc[0]).total_seconds() / ( 60 * 60) for v in df['datetime']]

In [11]:
df.set_index('datetime', inplace=True)

rain_resampled = df['rainPerInterval'].resample('3H').sum()
wind_resampled = df['windSpeed'].resample('3H').mean()


rain = rain_resampled.values
wind = wind_resampled.values

x_t = [1.5,4.5,7.5,10.5,13.5,16.5,19.5,22.5]
y_pos = [0,0,0,0,0,0,0,0]

df.reset_index(inplace=True)


/tmp/ipykernel_11691/2803122917.py:3: FutureWarning:

'H' is deprecated and will be removed in a future version, please use 'h' instead.

/tmp/ipykernel_11691/2803122917.py:4: FutureWarning:

'H' is deprecated and will be removed in a future version, please use 'h' instead.



In [12]:

def getImage(path, zoom=0.1):
    return OffsetImage(plt.imread(path), zoom=zoom)

paths_rain = []

for value in rain:
    if value/3 == 0:
        paths_rain.append('')
    elif 0 < value/3 < 2.5:
        paths_rain.append('LightRain.png')
    elif 2.5 <= value/3 <= 7.6:
        paths_rain.append('ModerateRain.png')
    elif value > 7.6:
        paths_rain.append('HeavyRain.png')

paths_wind = []

for value in wind:
    if value == 0:
        paths_wind.append('')
    elif 0 < value <= 10:
        paths_wind.append('LightWind.png')
    elif 10 < value <= 20:
        paths_wind.append('ModerateWind.png')
    elif 20 < value <= 30:
        paths_wind.append('StrongWind.png')    
    elif value > 30:
        paths_rain.append('VeryStrongWind.png')        


In [13]:


# Load the style sheet
plt.style.use('mints.mplstyle')

# Columns for the time series plots
time_series_columns = ['windSpeed','rainPerInterval', 'humidity', 'temperature', 'sol_el', 
                       'pm0_1', 'pm0_3', 'pm0_5', 'pm1_0', 'pm2_5', 'pm5_0', 'pm10_0']

# Titles for the subplots
titles = ['Wind Speed','Rainfall (mm/hr)', 'Humidity (%)', 'Temperature (°C)', 'Solar Elevation (degrees)',
          'PM 0.1 (µg/m³)', 'PM 0.3 (µg/m³)', 'PM 0.5 (µg/m³)', 'PM 1 (µg/m³)', 'PM 2.5 (µg/m³)',
          'PM 5 (µg/m³)', 'PM 10 (µg/m³)']


fig, axes = plt.subplots(nrows=len(time_series_columns), ncols=2, figsize=(20, 40),
                         gridspec_kw={'width_ratios': [1, 0.08],'height_ratios': [1, 1, 1, 1,1, 2, 2, 2, 2, 2, 2, 2], 'wspace': 0.01, 'hspace': 0.2})


def apply_custom_grid(ax):
    ax.grid(which='major', color='darkgrey', linewidth=2)
    ax.grid(which='minor', color='lightgrey', linewidth=1)


dt_min = df['dt'].min()
dt_max = np.ceil(df['dt'].max())

# Initialize lists to store min, max, and mid values
y_min = []
y_max = []
y_mid = []

date_label = f"{pd.to_datetime(df['datetime'].iloc[0]).strftime('%Y-%m-%d')} (local)"

colors = ['blue', 'blue', 'blue','red','yellow', 'grey', 'grey', 'grey', 'grey', 'grey', 'grey', 'grey', 'grey']


for i, (col, title) in enumerate(zip(time_series_columns, titles)):
    # Time series plot
    if col == 'rainPerInterval':
        axes[i,0].scatter(x_t, y_pos,marker='')
        for x0, y0, path in zip(x_t, y_pos,paths_rain):
         if path:  
             ab = AnnotationBbox(getImage(path), (x0, y0), frameon=False)
             axes[i, 0].add_artist(ab)   
         #ab = AnnotationBbox(getImage(path), (x0, y0), frameon=False)
         #axes[i,0].add_artist(ab)
        min_val = np.ceil(df[col].min())
        max_val = np.ceil(df[col].max())
        #axes[i, 0].set_yticks([min_val, max_val])
        y_min.append(min_val)
        y_max.append(max_val)
        axes[i, 0].axis('off')  # Turn off the axis for this subplot
        axes[i, 1].axis('off')  # Turn off the KDE plot axis for this subplot
        continue

    elif col == 'windSpeed':
        axes[i,0].scatter(x_t, y_pos,marker='')
        for x0, y0, path in zip(x_t, y_pos,paths_wind):
         if path:  
             ab = AnnotationBbox(getImage(path), (x0, y0), frameon=False)
             axes[i, 0].add_artist(ab)   
         #ab = AnnotationBbox(getImage(path), (x0, y0), frameon=False)
         #axes[i,0].add_artist(ab)
        min_val = np.ceil(df[col].min())
        max_val = np.ceil(df[col].max())
        #axes[i, 0].set_yticks([min_val, max_val])
        y_min.append(min_val)
        y_max.append(max_val)
        axes[i, 0].axis('off')  # Turn off the axis for this subplot
        axes[i, 1].axis('off')  # Turn off the KDE plot axis for this subplot
        continue

    elif col == 'humidity':
        sns.lineplot(x='dt', y=col, data=df, ax=axes[i, 0], color='blue', label=title)
        axes[i, 0].legend_.remove()
        min_val = np.ceil(df[col].min())
        max_val = np.ceil(df[col].max())
        axes[i, 0].set_yticks([min_val, max_val])
        y_min.append(min_val)
        y_max.append(max_val)
        
    elif col == 'temperature':
        sns.lineplot(x='dt', y=col, data=df, ax=axes[i, 0], color='red', label=title)
        # Add dew point to temperature plot
        sns.lineplot(x='dt', y='dewPoint', data=df, ax=axes[i, 0], color='blue', linestyle='--', label='Dew Point')
        min_val = np.ceil(df['dewPoint'].min())
        max_val = np.ceil(df[col].max())
        axes[i, 0].set_yticks([min_val, max_val])
        y_min.append(min_val)
        y_max.append(max_val)

    

    elif col == 'windAngle':
        sns.lineplot(x='dt', y=col, data=df, ax=axes[i, 0], color='green', label=title)
        axes[i, 0].legend_.remove()
        min_val = np.ceil(df[col].min())
        max_val = np.ceil(df[col].max())
        axes[i, 0].set_yticks([min_val, max_val])
        y_min.append(min_val)
        y_max.append(max_val)        
        
    elif col == 'sol_el':
        sns.lineplot(x='dt', y=col, data=df, ax=axes[i, 0], color='yellow', label=title)
        axes[i, 0].fill_between(df['dt'], df['sol_el'], color='yellow', alpha=0.2)
        axes[i, 0].set_ylim(0, 90)
        axes[i, 0].set_yticks([0, 90])
        axes[i, 0].legend_.remove()
        y_min.append(0)
        y_max.append(90)
        
    else:
        sns.lineplot(x='dt', y=col, data=df, ax=axes[i, 0], color='grey', label=title)
        
        min_val = np.floor(df[col].min()).astype(int)
        max_val = np.ceil(df[col].max()).astype(int)
        mid_val = (max_val + min_val) / 2
        axes[i, 0].set_yticks([min_val, mid_val, max_val])
        y_min.append(min_val)
        y_max.append(max_val)
        y_mid.append(mid_val)
        axes[i, 0].legend_.remove()

    axes[i, 0].set_title(title, fontsize=20, loc='left')
    
    
    if i == len(time_series_columns) - 1:
        axes[i, 0].set_xlabel(date_label, fontsize=20)
        axes[i, 0].set_ylabel('')
        axes[i, 0].xaxis.set_major_locator(MultipleLocator(3))
    else:
        axes[i, 0].set_xlabel('')
        axes[i, 0].xaxis.set_major_locator(MultipleLocator(3))
        axes[i, 0].set_xticklabels([])  
        axes[i, 0].set_ylabel('')

    axes[i, 0].tick_params(axis='x')  
    
    axes[i, 0].set_xlim(dt_min, dt_max)
    
    apply_custom_grid(axes[i, 0])

    sns.kdeplot(y=df[col], ax=axes[i, 1], color=colors[i], fill=True)
    axes[i, 1].set_ylabel('')
    axes[i, 1].set_xlabel('')
    axes[i, 1].set_yticklabels([])  
    axes[i, 1].set_xticklabels([])  
    axes[i, 1].set_ylim(y_min[i], y_max[i])

    axes[i, 1].grid(False)  
    axes[i, 1].spines['top'].set_visible(False)
    axes[i, 1].spines['right'].set_visible(False)
    axes[i, 1].spines['left'].set_visible(False)
    axes[i, 1].spines['bottom'].set_visible(False)

    axes[i, 1].axis('off')

plt.tight_layout()
plt.savefig('Daily.png', dpi=fig.dpi)
plt.show()



/tmp/ipykernel_11691/2162767244.py:157: UserWarning:

This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.



<Figure size 6000x12000 with 24 Axes>

In [14]:

sns.axes_style()
sns.set_style("darkgrid", {"axes.facecolor": "1.0", "font.family": "Montserrat"})

# Heatmap for particle counts
cols_to_keep = ['dt', 'pc0_1', 'pc0_3', 'pc0_5', 'pc2_5', 'pc1_0', 'pc5_0', 'pc10_0']
df_subset = df[cols_to_keep]

new_column_names = {
    'pc0_1': '0.1 µm',
    'pc0_3': '0.3 µm',
    'pc0_5': '0.5 µm',
    'pc2_5': '2.5 µm',
    'pc1_0': '1.0 µm',
    'pc5_0': '5.0 µm',
    'pc10_0': '10.0 µm'
}
df_subset.rename(columns=new_column_names, inplace=True)

df_melted = pd.melt(df_subset, id_vars=["dt"], var_name="Particle Count", value_name="Value")
#df_pivot = df_melted.pivot("Particle Count", "dt", "Value")
df_pivot = df_melted.pivot(index="Particle Count", columns="dt", values="Value")


desired_order = ['10.0 µm','5.0 µm','2.5 µm','1.0 µm','0.5 µm','0.3 µm','0.1 µm' ]
df_pivot = df_pivot.reindex(desired_order)

# Mask values less than or equal to zero
df_pivot[df_pivot <= 0] = np.nan

# Set the figure size to (12, 5)
plt.figure(figsize=(12, 5))
ax = sns.heatmap(np.log10(df_pivot), cmap="turbo", fmt="0.0f", alpha=1.0, 
                 cbar_kws={"label": "Log of Particle Count", "orientation": "vertical"})

x_tick_freq = 180  # Frequency of x-axis ticks
x_tick_locations = np.arange(0, len(df_pivot.columns), x_tick_freq)
x_tick_labels = [f'{label:.0f}' for label in df_pivot.columns[x_tick_locations]]  # Removing decimals

ax.set_xticks(x_tick_locations)
ax.set_xticklabels(x_tick_labels, rotation=0)  # Set rotation to 0 for horizontal labels

ax.tick_params(axis='both', which='both', bottom=False, top=False, left=False, right=False)
ax.grid(False)
plt.title("Heatmap of Particle Counts Over Time", fontsize=10, loc='left')
plt.xlabel(date_label)  # Assuming you are labeling x-axis as Datetime
plt.ylabel(" ")
plt.tight_layout()
plt.savefig('Dailyheatmap.png', dpi=fig.dpi)
plt.show()


/tmp/ipykernel_11691/2743182358.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



<Figure size 3600x1500 with 2 Axes>

\begin{center}
\includegraphics[width=\textwidth]{Daily.png}
\includegraphics[width=\textwidth]{Dailyheatmap.png}
\end{center}